In [1]:
pip install supabase

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)

   -- -------------------------------------  1/18 [propcache]
   ---- -----------------------------------  2/18 [pluggy]
   ------ ---------------------------------  3/18 [iniconfig]
   ----------- ----------------------------  5/18 [hpack]
   ----------- ----------------------------  5/18 [hpack]
  Attempting uninstall: aiohappyeyeballs
   ----------- ----------------------------  5/18 [hpack]
    Found existing installation: aiohappyeyeballs 2.4.0
   ----------- ----------------------------  5/18 [hpack]
    Uninstalling aiohappyeyeballs-2.4.0:
   ----------- ----------------------------  5/18 [hpack]
      Successfully uninstalled aiohappyeyeballs-2.4.0
   ----------- ----------------------------  5/18 [hpack]
   ------------- --------------------------  6/18 [aiohappyeyeballs]
  Attempting uninstall: yarl
   ------------- -------------------------- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.28 requires langchain-core<0.2.0,>=0.1.31, but you have langchain-core 0.3.64 which is incompatible.
langchain-community 0.0.28 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.3.45 which is incompatible.
langchain-community 0.0.28 requires tenacity<9.0.0,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
from supabase import create_client, Client
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [ ]:
response = (
    supabase.table("documents")
    .select("*")
    .execute()
)

In [3]:
print(response)

data=[{'id': 13, 'content': 'Raphe mPhibr raises $100 mn to expand aircraft design & manufacturing in India, The Economic Times B2B\nAdvt\n\nJoin the community of 2M+ industry professionals. Subscribe to Newsletter to get latest insights & analysis in your inbox. Get updates on your preferred social platform Follow us for the latest news, insider access to events and more.\n\nIndian drone startup Raphe mPhibr on Thursday announced that it has raised $100 million in a round led by General Catalyst, with participation from existing investors, including Amal Parikh, Think Investments, and other prominent high-net-worth family offices.This is the largest round of private funding raised by an Indian company in this industry to date and takes Raphe mPhibr’s total capital raised to $145 million.“We are proud to join forces with Raphe mPhibr on their transformative journey to redefine engineering-led manufacturing across a spectrum of critical applications,” said Neeraj Arora, Managing Directo

In [4]:
from sentence_transformers import SentenceTransformer

c:\Users\meyas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Initializing and testing supabase if it performs sql type operations.

In [11]:
content = "Hi my name is YASH."
embedding = model.encode(content).tolist()

In [14]:
# to delete a record
response = supabase.table("documents").delete().eq("id", 1).execute()


In [ ]:
# to insert a record
response = supabase.table("documents").insert({
    "content": content,
    "embedding": embedding
}).execute()

print(response)

data=[{'id': 3, 'content': 'Hi my name is YASH.', 'embedding': '[-0.05704804,-0.04335864,0.025902646,-0.026240876,-0.07127375,-0.052402675,0.12980427,-0.032224145,0.05129746,0.032655317,-0.004520648,-0.044974297,-0.00062160497,-0.037785318,0.06817682,-0.004310294,0.045492567,-0.022800608,-0.08162923,-0.04024407,0.008971929,0.04570287,-0.051916663,0.004828336,-0.061018992,-0.057125438,0.009890478,0.13997774,-0.03531821,-0.05151729,-0.00063081284,-0.0021875591,0.023476448,0.06895654,0.01047872,0.033084974,-0.012362848,-0.04017081,-0.033662733,0.051565297,-0.03330429,-0.05245864,-0.041536562,0.025890408,0.010297545,0.008164345,0.0023859306,0.022965558,0.050628845,0.013669273,-0.10296366,-0.075015046,-0.017834656,0.060609005,0.09008978,-0.029558241,-0.04767084,0.0028964581,-0.0015504101,0.03926501,-0.07821904,0.059728853,-0.09731708,-0.03764251,0.007857799,-0.01759093,-0.013672034,0.0034175958,-0.0023357656,-0.01477504,-0.029458938,-0.018473936,0.06427542,0.023674665,-0.0053053903,-0.09090

In [19]:
#inserting multiple records
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]

for i,content in enumerate(corpus):
    supabase.table('documents').insert({
       "content": content,
       "embedding": model.encode(content).tolist() 
    }).execute()

### Extracting news data and passing it to the vector DB

In [7]:
import feedparser
from newspaper import Article
from datetime import datetime

In [8]:
rss_url_econ = "https://b2b.economictimes.indiatimes.com/rss/recentstories"
feed = feedparser.parse(rss_url_econ)

print(f"\n*** ET News ({len(feed.entries)} articles):\n")

corpus = []
for entry in feed.entries:
    url = entry.link
    published = entry.published_parsed
    pub_date = datetime(*published[:6])
    
    article = Article(url)
    article.download()
    article.parse()
    
    print(f"Title: {article.title}")
    print(f"URL: {url}")

    # adding article title and text to corpus
    data = article.title + '\n' + article.text
    corpus.append(data)
# creating embedding of article title and text and pushing to vector DB
for i,content in enumerate(corpus):
    supabase.table('documents').insert({
       "content": content,
       "embedding": model.encode(content).tolist() 
    }).execute()



*** ET News (20 articles):

Title: Early Nykaa investor set to offload stake worth $150 million via block deal, The Economic Times B2B
URL: https://b2b.economictimes.indiatimes.com/news/entrepreneur/early-nykaa-investor-set-to-offload-stake-worth-150-million-via-block-deal/122219439
Title: Jeff Bezos Amazon Shares Sale: Jeff Bezos Sells $737 Million in Amazon Shares as Fourth-Richest Person on Earth, The Economic Times B2B
URL: https://b2b.economictimes.indiatimes.com/news/entrepreneur/jeff-bezos-sells-737-million-in-amazon-shares-as-fourth-richest-person-on-earth/122196139
Title: Zomato Founder Deepinder Goyal Ventures into Regional Aviation with LAT Aerospace, The Economic Times B2B
URL: https://b2b.economictimes.indiatimes.com/news/entrepreneur/zomato-founder-deepinder-goyal-ventures-into-regional-aviation-with-lat-aerospace/122154201
Title: Anant Ambani Salary: Anant Ambani's Lucrative ₹10-20 Crore Salary as Executive Director of Reliance Industries, The Economic Times B2B
URL: ht

NameError: name 'supabase' is not defined

In [ ]:
# text representation in each article
url = "https://b2b.economictimes.indiatimes.com/news/raphe-mphibr-raises-100-mn-to-expand-aircraft-design-manufacturing-in-india/122086129"

article = Article(url)
article.download()
article.parse()
print(article.title)
print(article.publish_date)
print(article.text)


Raphe mPhibr raises $100 mn to expand aircraft design & manufacturing in India, The Economic Times B2B
None
Advt

Join the community of 2M+ industry professionals. Subscribe to Newsletter to get latest insights & analysis in your inbox. Get updates on your preferred social platform Follow us for the latest news, insider access to events and more.

Indian drone startup Raphe mPhibr on Thursday announced that it has raised $100 million in a round led by General Catalyst, with participation from existing investors, including Amal Parikh, Think Investments, and other prominent high-net-worth family offices.This is the largest round of private funding raised by an Indian company in this industry to date and takes Raphe mPhibr’s total capital raised to $145 million.“We are proud to join forces with Raphe mPhibr on their transformative journey to redefine engineering-led manufacturing across a spectrum of critical applications,” said Neeraj Arora, Managing Director at General Catalyst.“What s

In [ ]:
#human query and vector DB semantic search
query = "Adani News"
query_embedding = model.encode(query).tolist()

In [24]:
print(query_embedding)

[-0.03192391246557236, 0.021750235930085182, -0.012451724149286747, 0.10700082033872604, 0.015379457734525204, -0.054279204457998276, 0.08965086936950684, -0.017536716535687447, -0.0780726820230484, 0.013045559637248516, -0.03484673425555229, 0.02355976030230522, -0.06169775873422623, 0.0034138767514377832, -0.029459381476044655, 0.04450124129652977, -0.026491597294807434, 0.01587028056383133, -0.02835891768336296, 0.013719542883336544, -0.035908397287130356, -0.023345468565821648, -0.025480087846517563, 0.04626033455133438, 0.04216928407549858, 0.009811494499444962, -0.013179328292608261, -0.026168078184127808, -0.017399579286575317, -0.13073666393756866, -0.015976978465914726, -0.058218710124492645, 0.13407649099826813, 0.0022757737897336483, -0.0311322882771492, -0.011843584477901459, -0.004244055598974228, -0.042029689997434616, -0.039054226130247116, 0.013474266976118088, 0.0337214395403862, -0.060453861951828, -0.04458828270435333, -0.07483147084712982, 0.035719309002161026, -0.0

In [26]:
# Call the 'match_documents' RPC
response = supabase.rpc("match_documents", {
    "query_embedding": query_embedding,
    "match_threshold": 0.5,
    "match_count": 10
}).execute()

# Access the returned data
documents = response.data

for doc in documents:
    print(f"Content: {doc['content']}\n")

Content: Adani Group Investment: Adani Group to invest $100 billion over five years in clean energy sector, The Economic Times B2B
Advt

US Indictment

Join the community of 2M+ industry professionals. Subscribe to Newsletter to get latest insights & analysis in your inbox. Get updates on your preferred social platform Follow us for the latest news, insider access to events and more.

The Adani Group is targeting 100 GW energy generation capacity across thermal, renewable and pumped hydropower by 2030, chairman Gautam Adani said on Tuesday. This will include 31 GW capacity from Adani Power and 45 GW renewable energy from Adani Green Energy Group company Adani Energy Solutions , meanwhile, is implementing transmission projects worth ₹44,000 crore and smart metering projects of ₹13,600 crore, Adani said at an annual general meeting of shareholders of the group's flagship company Adani Enterprises The infrastructure and utilities conglomerate recorded a 7 per cent rise in revenue to ₹2.72

### Azure AI foundry embedding (text-embedding-3-small) example

In [12]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [13]:
embed_key = os.getenv("EMBED_KEY")

In [18]:
import os
from openai import AzureOpenAI

endpoint = "https://yash-mcp-ai.cognitiveservices.azure.com/"
model_name = "text-embedding-3-small"
deployment = "text-embedding-3-small"

api_version = "2024-02-01"

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    api_key = embed_key,
    azure_endpoint= endpoint
)

response = client.embeddings.create(
    input=["first phrase","second phrase","third phrase"],
    model=deployment
)

for item in response.data:
    length = len(item.embedding)
    print(
        f"data[{item.index}]: length={length}, "
        f"[{item.embedding[0]}, {item.embedding[1]}, "
        f"..., {item.embedding[length-2]}, {item.embedding[length-1]}]"
    )
print(response.usage)

data[0]: length=1536, [-0.007196240127086639, 0.007499529980123043, ..., 0.01614919863641262, -0.004848698154091835]
data[1]: length=1536, [-0.00304086459800601, 0.009223765693604946, ..., 0.029916152358055115, 0.020982539281249046]
data[2]: length=1536, [-0.013793058693408966, 0.03190053254365921, ..., 0.01751914620399475, 0.022650687023997307]
Usage(prompt_tokens=6, total_tokens=6)


### AI foundry llm model testing